# Lecture 2: Code specialization

In lecture 1, we learned how the semantics of types affect performance — it was not *possible* to implement a C-speed `sum` function, for example, unless (a) the element type is attached to the array as a whole so that (b) the element data can be stored consecutively in memory and (b) the computional code can be specialized at compile-time for `+` on this element type.

For code to be fast, however, it must also be possible to take advantage of this information in your own code.  This is obviously possible in statically typed languages like C, which are compiled to machine instructions ahead of time with type information declared explicitly to the compiler.  It is more subtle in a language like Julia, where the programmer does *not* usually explicitly declare type information.

In particular, we will talk about how Julia achieves both fast code and *type-generic code* (e.g. a `sum` function that works on any container of any type supporting `+`, even user-defined types), by aggressive automated **code specialization** by the Julia compiler.

## A type-generic sum function

Let's again look at our simple hand-written `sum` function.

In [1]:
function mysum(a)
    s = zero(eltype(a))
    for x in a
        s += x
    end
    return s
end

mysum (generic function with 1 method)

In [2]:
a = rand(10^7)
mysum(a)

5.000392958354606e6

In [3]:
mysum(a) ≈ sum(a)

true

In [4]:
using BenchmarkTools, Compat
@btime sum($a)
@btime mysum($a)

  4.633 ms (0 allocations: 0 bytes)
  11.374 ms (0 allocations: 0 bytes)


5.000392958354606e6

Hooray!  Basically the same speed as Julia's built-in `sum` function and `numpy.sum`!   And it only required **7 lines of code**, some care with types, and a very minor bit of wizardry with the `@simd` tag to get the last factor of two.

Moreover, the code is still **type generic**: it can sum any container of any type that works with addition.   For example, it works for complex numbers, which are about two times slower as you might expect (since each complex addition requires two real additions):

In [5]:
z = rand(Complex{Float64}, length(a));
@btime mysum($z)

  11.909 ms (0 allocations: 0 bytes)


4.998987382599792e6 + 5.000017302742645e6im

And we **didn't have to declare any types** of any arguments or variables; the compiler figured everything out.  How?

In [6]:
s = Set([2, 17, 6 , 24])

Set([2, 17, 6, 24])

In [7]:
typeof(s)

Set{Int64}

In [8]:
2 in s, 13 in s

(true, false)

In [9]:
mysum(s)

49

## Type inference and specialization

To go any further, you need to understand something very basic about how Julia works.  Suppose we define a very simple function:

In [10]:
f(x) = x + 1

f (generic function with 1 method)

We didn't declare the type of `x`, and so our function `f(x)` will work with **any type of `x`** (as long as the `+ 1` operation is defined for that type):

In [11]:
f(3) # x is an integer (technically, a 64-bit integer)

4

In [12]:
@btime f(3)

  1.598 ns (0 allocations: 0 bytes)


4

In [13]:
@benchmark f(3)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.598 ns (0.00% GC)
  median time:      1.922 ns (0.00% GC)
  mean time:        2.048 ns (0.00% GC)
  maximum time:     409.245 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [14]:
f(3.1) # x is a floating-point value (Float64)

4.1

In [15]:
f([1,2,3]) # x is an array of integers

3-element Array{Int64,1}:
 2
 3
 4

In [16]:
@time f(0x01)

  0.003129 seconds (207 allocations: 12.979 KiB)


2

In [17]:
@time f(0x01)

  0.000004 seconds (4 allocations: 160 bytes)


2

In [18]:
@time f(0x01)

  0.000002 seconds (4 allocations: 160 bytes)


2

In [19]:
f("hello")

LoadError: [91mMethodError: no method matching +(::String, ::Int64)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +([91m::Complex{Bool}[39m, ::Real) at complex.jl:247
  +([91m::Char[39m, ::Integer) at char.jl:40
  ...[39m

How can a function like `f(x)` work for any type?  In Python, `x` would be a "box" that could contain anything, and it would then look up *at runtime* how to compute `x + 1`.  But we just saw that untyped Julia `sum` code could be fast.

The secret is **just-in-time (JIT) compilation**.   The first time you call `f(x)` **with a new type of argument** `x`, it will **compile a new version of `f` specialized for that type**.  The *next* time it calls `f(x)` with the same argument type

So, right now, after evaluating the above code, we have *three* versions of `f` compiled and sitting in memory: one for `x` of type `Int` (we say `x::Int` in Julia), one for `x::Float64`, and one for `x::Vector{Int}`.

We can even see what the compiled code for `f(x::Int)` looks like, either the [compiler (LLVM) bytecode](https://en.wikipedia.org/wiki/LLVM) or the low-level (below C!) [assembly code](https://en.wikipedia.org/wiki/Assembly_language):

In [20]:
@code_llvm f(1)


define i64 @julia_f_61345(i64) #0 !dbg !5 {
top:
  %1 = add i64 %0, 1
  ret i64 %1
}


In [21]:
@code_native f(1)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[10]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	leaq	1(%rdi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Let's break this down.  When you tell Julia's compiler that `x` is an `Int`, it:

* It knows `x` fits into a 64-bit CPU register (and is passed to the function via a register).

* Looks at `x + 1`.  Since `x` and `1` are both `Int`, it knows it should call the `+` function for two `Int` values.  This corresponds to *one machine instruction* `leaq` to add two 64-bit registers.

* Since the `+` function here is so simple, it won't bother to do a function call.  It will [inline](https://en.wikipedia.org/wiki/Inline_expansion) the `(+)(Int,Int)` function into the compiled `f(x)` code.

* Since it now knows what `+` function it is calling, it knows that the *result* of the `+` is *also* an `Int`, and it can return it via register.

This process works recursively if we define a new function `g(x)` that calls `f(x)`:

In [22]:
g(x) = f(x) * 2
g(1)

4

In [23]:
@code_llvm g(1)


define i64 @julia_g_61588(i64) #0 !dbg !5 {
top:
  %1 = shl i64 %0, 1
  %2 = add i64 %1, 2
  ret i64 %2
}


When it specialized `g` for `x::Int`, it not only figured out what `f` function to call, it not only *inlined* `f(x)` *into `g`*, but the compiler was smart enough to *combine the two additions* into a single addition `x + 4`.

In [24]:
h(x) = g(x) * 2
@code_llvm h(1)


define i64 @julia_h_61590(i64) #0 !dbg !5 {
top:
  %1 = shl i64 %0, 2
  %2 = add i64 %1, 4
  ret i64 %2
}


Julia's type inference is smart enough that it can figure out the return type even for recursive functions:

In [25]:
fib(n::Integer) = n < 3 ? 1 : fib(n-1) + fib(n-2)

fib (generic function with 1 method)

In [26]:
[fib(n) for n = 1:10]

10-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13
 21
 34
 55

In [27]:
fib.(1:10)

10-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13
 21
 34
 55

In [28]:
@code_warntype fib(1)

Variables:
  #self#::#fib
  n::Int64

Body:
  begin 
      unless (Base.slt_int)(n::Int64, 3)::Bool goto 3
      return 1
      3: 
      SSAValue(1) = $(Expr(:invoke, MethodInstance for fib(::Int64), :(Main.fib), :((Base.sub_int)(n, 1)::Int64)))
      SSAValue(0) = $(Expr(:invoke, MethodInstance for fib(::Int64), :(Main.fib), :((Base.sub_int)(n, 2)::Int64)))
      return (Base.add_int)(SSAValue(1), SSAValue(0))::Int64
  end::Int64


In [29]:
3

3

In [30]:
typeof(3)

Int64

## Dispatch on the argument type

It is often useful to declare the argument types in Julia.  For example, above, we define `fib(n::Integer)`.  This says that the argument must be *some* type of integer.  If we give it a different number type, it will now give an error:

In [31]:
fib(3.7)

LoadError: [91mMethodError: no method matching fib(::Float64)[0m
Closest candidates are:
  fib([91m::Integer[39m) at In[25]:1[39m

`Integer` is an "abstract" type in Julia.  There are many subtypes of `Integer` in Julia, including `Int64` (64-bit signed integers), `UInt8` (8-bit unsigned integers), and `BigInt` (arbitrary-precision integers: the number of digits grows as the numbers get larger and larger).

Declaring the argument type has **no effect on performance** in Julia: the compiler automatically specializes the function when it is called, even if we declare no type at all.  There are three main reasons to declare an argument type in Julia:

* *Clarity*: Declaring the argument type can help readers understand the code.  (However, *over-specifying* the type may make the function less general than it has to be!)

* *Correctness*: Our `fib` function above would have given *some* answer if we allowed the user to pass `3.7`, but it probably wouldn't be the intended answer.

* *Dispatch*: Julia allows you to define **different versions of a function** (different **methods**) for different argument types.

For example, let's use this to define two versions of a [factorial](https://en.wikipedia.org/wiki/Factorial) function `myfact`, one for integers that recursively multiplies its arguments:

In [32]:
function myfact(n::Integer) 
    n < 0 && throw(DomainError("n must be positive"))
    return n < 2 ? one(n) : n * myfact(n-1)
end

myfact (generic function with 1 method)

In [33]:
myfact(10)

3628800

You need `BigInt` for factorials of larger arguments, since factorials grow rapidly (faster than exponentially with `n`):

In [34]:
myfact(BigInt(100))

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

Since we said `n::Integer`, this will give an error for a floating-point argument:

In [35]:
myfact(3.7)

LoadError: [91mMethodError: no method matching myfact(::Float64)[0m
Closest candidates are:
  myfact([91m::Integer[39m) at In[32]:2[39m

However, it turns out that there is a very natural extension of the factorial to arbitrary real and complex numbers, via the [gamma function](https://en.wikipedia.org/wiki/Gamma_function): $n! = \Gamma(n+1)$.  Julia has a built-in `gamma(x)` function, so we can define `myfact` for other number types based on that:

In [36]:
myfact(x::Number) = gamma(x+1)

myfact (generic function with 2 methods)

In [37]:
myfact(5.0)

120.0

The "factorial" of $-\frac{1}{2}$ is then $\sqrt{\pi}$:

In [38]:
myfact(-0.5)^2

3.1415926535897936

Now there are two different *methods* of `myfact`.  You can get a list of the methods of any function by calling `methods`:

In [39]:
methods(myfact)

# 2 methods for generic function "myfact":
myfact(n::Integer) in Main at In[32]:2
myfact(x::Number) in Main at In[36]:1

## Multiple dispatch

A key property of Julia is that is based around the principle of [multiple dispatch](https://en.wikipedia.org/wiki/Multiple_dispatch): when you call a function `f(arguments...)` in Julia, it picks the **most specific method of f** based on **the types of *all* of the arguments**.

This can be thought of as a generalization of object-oriented programming (OOP).  In an OOP language like Python, you would typically write `object.method(arguments...)`: based on the type of `object`, it will decide which version of `method` to call (it "dispatches" to the correct `method`).  The Julia analogue is `method(object, arguments...)`.  But whereas an OOP language would only look at the type of `object` (*single* dispatch), Julia looks at both the types of `object` *and* the other `arguments` (*multiple* dispatch).

Multiple dispatch is very natural for talking about mathematical operations like `a + b`, which is just a call to the function `+` in Julia.   In an OOP language like C++, `a + b` decides what `+` function to call based on the type of `a`, which is rather weird: the function is "owned" by its first argument.  In Julia, it looks at both arguments, and there are in fact as huge number of `+` functions for different argument types:

In [40]:
methods(+)

# 180 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) in Base at complex.jl:232
+(x::Bool, y::Bool) in Base at bool.jl:89
+(x::Bool) in Base at bool.jl:86
+(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:96
+(x::Bool, z::Complex) in Base at complex.jl:239
+(a::Float16, b::Float16) in Base at float.jl:372
+(x::Float32, y::Float32) in Base at float.jl:374
+(x::Float64, y::Float64) in Base at float.jl:375
+(z::Complex{Bool}, x::Bool) in Base at complex.jl:233
+(z::Complex{Bool}, x::Real) in Base at complex.jl:247
+(x::Char, y::Integer) in Base at char.jl:40
+(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:312
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:334
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:327
+(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:321
+(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:289
+(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:346
+(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:362
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:460
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:453
+(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:447
+(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:308
+(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:277
+(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:284
+(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:292
+(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:300
+(B::BitArray{2}, J::UniformScaling) in Base.LinAlg at linalg/uniformscaling.jl:59
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:87
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:135
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:197
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) in Base.Pkg.Resolve.MaxSum.FieldValues at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) in Base.Dates at dates/periods.jl:349
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) in Base.Dates at dates/periods.jl:347
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) in Base.Dates at dates/periods.jl:387
+(x::Date, y::Base.Dates.Day) in Base.Dates at dates/arithmetic.jl:77
+(x::Date, y::Base.Dates.Week) in Base.Dates at dates/arithmetic.jl:75
+(dt::Date, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:58
+(dt::Date, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:32
+(dt::Date, t::Base.Dates.Time) in Base.Dates at dates/arithmetic.jl:20
+(t::Base.Dates.Time, dt::Date) in Base.Dates at dates/arithmetic.jl:24
+(x::Base.Dates.Time, y::Base.Dates.TimePeriod) in Base.Dates at dates/arithmetic.jl:81
+(dt::DateTime, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:52
+(dt::DateTime, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:28
+(x::DateTime, y::Base.Dates.Period) in Base.Dates at dates/arithmetic.jl:79
+(y::AbstractFloat, x::Bool) in Base at bool.jl:98
+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:32
+(x::Integer, y::Ptr) in Base at pointer.jl:128
+(z::Complex, w::Complex) in Base at complex.jl:221
+(z::Complex, x::Bool) in Base at complex.jl:240
+(x::Real, z::Complex{Bool}) in Base at complex.jl:246
+(x::Real, z::Complex) in Base at complex.jl:258
+(z::Complex, x::Real) in Base at complex.jl:259
+(x::Rational, y::Rational) in Base at rational.jl:245

You can use the `@which` method to see which method is called for a specific set of arguments:

In [41]:
@which (+)(3,4)

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:32

Here is a somewhat artificial example where we define three methods for `f(x,y)` depending on the argument types, and we can see how Julia picks which one to call:

In [42]:
f(x,y) = 3
f(x::Integer,y::Integer) = 4
f(x::Number,y::Integer) = 5

f (generic function with 4 methods)

In [43]:
f(3,4), f(3.5,4), f(4,3.5)

(4, 5, 3)

One nice thing about Julia's "verb-centric" multiple-dispatch approach (as opposed to OOP's "noun-centric" approach) is that we can add new methods and functions to existing types (unlike OOP where you need to create a subclass to add new methods).

For example, string concatenation in Julia is performed by the `*` operator:

In [44]:
"3" * "7"

"37"

The `+` operator is not defined for strings:

In [45]:
"3" + "7"

LoadError: [91mMethodError: no method matching +(::String, ::String)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424[39m

But we can define it if we want!  Here, instead of defining our own function like `myfact`, we are adding a new method of an *existing* function that was defined in `Base` (Julia's standard library).  So, we need to explicitly tell Julia that we are extending `Base.+`, not defining a new function that happens to be called `+`:

In [46]:
Base.:+(a::AbstractString, b::AbstractString) = a * " + " * b

In [47]:
"3" + "7"

"3 + 7"

Now that we have defined `+`, we can even sum arrays of strings, because `sum` works for anything that defines `+` (and `zero` for summing empty arrays):

In [48]:
sum(["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog."])

"The + quick + brown + fox + jumps + over + the + lazy + dog."

## Defeating type inference: Type instabilities

To get good performance, there are some fairly simple rules that you need to follow in Julia code to avoid defeating the compiler's type inference.   See also the [performance tips section of the Julia manual](http://docs.julialang.org/en/stable/manual/performance-tips/).

Three of the most important are:

* Don't use (non-constant) global variables in critical code — put your critical code into a function (this is good advice anyway, from a software-engineering standpoint).  The compiler assumes that a **global variable can change type at any time**, so it is always stored in a "box", and "taints" anything that depends on it.

* Local variables should be "type-stable": **don't change the type of a variable inside a function**.  Use a new variable instead.

* Functions should be "type-stable": **a function's return type should only depend on the argument types, not on the argument values**.

To diagnose all of these problems, the `@code_warntype` macro that we used above is your friend.  If it labels any variables (or the function's return value) as `Any` or `Union{...}`, it means that the compiler couldn't figure out a precise type.

The third point, type-stability of functions, leads to lots of important but subtle choices in library design.  For example, consider the (built-in) `sqrt(x)` function, which computes $\sqrt{x}$:

In [49]:
sqrt(4)

2.0

You might think that `sqrt(-1)` should return $i$ (or `im`, in Julia syntax).  (Matlab's `sqrt` function does this.)  Instead, we get:

In [50]:
sqrt(-1)

LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).

In [51]:
sqrt(-1 + 0im)

0.0 + 1.0im

Why did Julia implement `sqrt` in this silly way, throwing an error for negative arguments unless you add a zero imaginary part?  Any reasonable person wants an imaginary result from `sqrt(-1)`, surely?

The problem is that defining `sqrt` to return an imaginary result from `sqrt(-1)` would **not be type stable**: `sqrt(x)` would return a real result for non-negative real `x`, and a complex result for negative real `x`, so the **return type would depend on the value of `x`** and **not just its type.**

That would defeat type inference, not just for the `sqrt` function, but for **anything the sqrt function touches**.  Unless the compiler can somehow figure out `x ≥ 0`, it will have to either store the result in a "box" or compile two branches of the result.  Let's see how that works by defining our own square-root function:

In [52]:
mysqrt(x::Complex) = sqrt(x)
mysqrt(x::Real) = x < 0 ? sqrt(complex(x)) : sqrt(x)

mysqrt (generic function with 2 methods)

This definition is an example of Julia's [multiple dispatch style](http://docs.julialang.org/en/stable/manual/methods/), which in some sense is a generalization of object-oriented programming but focuses on "verbs" (functions) rather than nouns.  We will discuss this more in a later lecture.

The `::Complex` and `::Real` are argument-type declarations.  Such declarations are **not related to performance**, but instead **act as a "filter"** to allow us to have one version of `mysqrt` for complex arguments and another for real arguments.

In [53]:
mysqrt(2)

1.4142135623730951

In [54]:
mysqrt(-2)

0.0 + 1.4142135623730951im

In [55]:
mysqrt(-2+0im)

0.0 + 1.4142135623730951im

Looks great, right?  But let's see what happens to type inference in a function that calls `mysqrt` instead of `sqrt`:

In [56]:
slowfun(x) = mysqrt(x) + 1
@code_warntype slowfun(2)

Variables:
  #self#::#slowfun
  x::Int64
  #temp#@_3::Union{Complex{Float64}, Float64}
  #temp#@_4::Core.MethodInstance
  #temp#@_5::Union{Complex{Float64}, Float64}

Body:
  begin 
      $(Expr(:inbounds, false))
      # meta: location In[52] mysqrt 2
      unless (Base.slt_int)(x::Int64, 0)::Bool goto 6
      #temp#@_3::Union{Complex{Float64}, Float64} = $(Expr(:invoke, MethodInstance for sqrt(::Complex{Float64}), :(Base.sqrt), :($(Expr(:new, Complex{Float64}, :((Base.sitofp)(Float64, x)::Float64), :((Base.sitofp)(Float64, 0)::Float64))))))
      goto 8
      6: 
      #temp#@_3::Union{Complex{Float64}, Float64} = (Base.Math.sqrt_llvm)((Base.sitofp)(Float64, x::Int64)::Float64)::Float64
      8: 
      # meta: pop location
      $(Expr(:inbounds, :pop))
      unless (#temp#@_3::Union{Complex{Float64}, Float64} isa Complex{Float64})::Bool goto 14
      #temp#@_4::Core.MethodInstance = MethodInstance for +(::Complex{Float64}, ::Int64)
      goto 23
      14: 
      unless (#temp#@_3::U

Because the compiler **doesn't know at compile-time that x is positive** (at compile-time it **uses only types, not values**, it doesn't know whether the result is real (`Float64`) or complex (`Complex{Float64}`) and has to store it in a "box".  This kills performance.

## Defining our own types

Let's define our own type to represent a **"point" in two dimensions**.  Each point will have an $(x,y)$ location.  So that we can use the points with our `sum` functions above, we'll also define `+` and `zero` functions to do the obvious **vector addition**.

One such definition in Julia is:

In [57]:
mutable struct Point1
    x
    y
end
Base.:+(p::Point1, q::Point1) = Point1(p.x + q.x, p.y + q.y)
Base.zero(::Type{Point1}) = Point1(0,0)

Point1(3,4)

Point1(3, 4)

In [58]:
Point1(3,4) + Point1(5,6)

Point1(8, 10)

Our type is very generic, and can hold any type of `x` and `y` values:

In [59]:
Point1(3.7, 4+5im)

Point1(3.7, 4 + 5im)

Perhaps too generic:

In [60]:
Point1("x", [3,4,5])

Point1("x", [3, 4, 5])

Since `x` and `y` can be *anything*, they must be **pointers to "boxes"**.  This is **bad news for performance**.

A `mutable struct` is *mutable*, which means we can create a `Point1` object and then change `x` or `y`:

In [61]:
p = Point1(3,4)
p.x = 7
p

Point1(7, 4)

This means that every reference to a `Point1` object must be a *pointer* to an object stored elsewhere in memory, because *how else would we "know" when an object changes?*  Furthermore, an **array of `Point1` objects must be an array of pointers** (which is **bad news for performance** again):

In [62]:
P = [p,p,p]

3-element Array{Point1,1}:
 Point1(7, 4)
 Point1(7, 4)
 Point1(7, 4)

In [63]:
p.y = 8
P

3-element Array{Point1,1}:
 Point1(7, 8)
 Point1(7, 8)
 Point1(7, 8)

Let's test this out by creating an array of `Point1` objects and summing it.  Ideally, this would be about twice as slow as summing an equal-length array of numbers, since there are twice as many numbers to sum.  But because of all of the boxes and pointer-chasing, it should be far slower.

To create the array, we'll call the `Point1(x,y)` constructor with our array `a`, using Julia's ["dot-call" syntax](http://docs.julialang.org/en/stable/manual/functions/#dot-syntax-for-vectorizing-functions) that applies a function "element-wise" to arrays:

In [64]:
a1 = Point1.(a, a)

10000000-element Array{Point1,1}:
 Point1(0.106974, 0.106974)  
 Point1(0.188845, 0.188845)  
 Point1(0.484844, 0.484844)  
 Point1(0.885487, 0.885487)  
 Point1(0.33817, 0.33817)    
 Point1(0.663541, 0.663541)  
 Point1(0.906624, 0.906624)  
 Point1(0.601869, 0.601869)  
 Point1(0.788574, 0.788574)  
 Point1(0.75761, 0.75761)    
 Point1(0.225378, 0.225378)  
 Point1(0.234817, 0.234817)  
 Point1(0.403443, 0.403443)  
 ⋮                           
 Point1(0.410147, 0.410147)  
 Point1(0.0750395, 0.0750395)
 Point1(0.060309, 0.060309)  
 Point1(0.307107, 0.307107)  
 Point1(0.559202, 0.559202)  
 Point1(0.498393, 0.498393)  
 Point1(0.663843, 0.663843)  
 Point1(0.441784, 0.441784)  
 Point1(0.331159, 0.331159)  
 Point1(0.0522143, 0.0522143)
 Point1(0.380872, 0.380872)  
 Point1(0.985776, 0.985776)  

In [65]:
@btime sum($a1)

  584.454 ms (29999997 allocations: 610.35 MiB)


Point1(5.0003929583536135e6, 5.0003929583536135e6)

In [66]:
@btime mysum($a1)

  548.308 ms (30000001 allocations: 610.35 MiB)


Point1(5.000392958354606e6, 5.000392958354606e6)

The time is at least **50× slower** than we would like, but consistent with our other timing results on "boxed" values from last lecture.

### An imperfect solution: A concrete immutable type

We can avoid these two problems by:

* Declare the types of `x` and `y` to be *concrete* types, so that they don't need to be pointers to boxes.
* Declare our Point to be an [immutable](https://en.wikipedia.org/wiki/Immutable_object) type (`x` and `y` cannot change), so that Julia is not forced to make every reference to a Point into a pointer: just `struct`, not `mutable struct`:

In [67]:
struct Point2
    x::Float64
    y::Float64
end
Base.:+(p::Point2, q::Point2) = Point2(p.x + q.x, p.y + q.y)
Base.zero(::Type{Point2}) = Point2(0.0,0.0)

Point2(3,4)

Point2(3.0, 4.0)

In [68]:
Point2(3,4) + Point2(5,6)

Point2(8.0, 10.0)

In [69]:
p = Point2(3,4)
P = [p,p,p]

3-element Array{Point2,1}:
 Point2(3.0, 4.0)
 Point2(3.0, 4.0)
 Point2(3.0, 4.0)

In [70]:
p.x = 6 # gives an error since p is immutable

LoadError: [91mtype Point2 is immutable[39m

If this is working as we hope, then summation should be much faster:

In [71]:
a2 = Point2.(a,a)
@btime sum($a2)

  12.728 ms (0 allocations: 0 bytes)


Point2(5.0003929583536135e6, 5.0003929583536135e6)

Now the time is **only about 10ms**, only slightly more than twice the cost of summing an array of individual numbers of the same length!

Unfortunately, we paid a big price for this performance: our `Point2` type only works with *a single numeric type* (`Float64`), much like a C implementation.

### The best of both worlds: Parameterized immutable types

How do we get a `Point` type that works for *any* type of `x` and `y`, but at the same time allows us to have an array of points that is concrete and homogeneous (every point in the array is forced to be the same type)?  At first glance, this seems like a contradiction in terms.

The answer is not to define a *single* type, but rather to **define a whole family of types** that are *parameterized* by the type `T` of `x` and `y`.  In computer science, this is known as [parametric polymorphism](https://en.wikipedia.org/wiki/Parametric_polymorphism).  (An example of this can be found in [C++ templates](https://en.wikipedia.org/wiki/Template_%28C%2B%2B%29).)

In Julia, we will define such a family of types as follows:

In [72]:
struct Point3{T<:Real}
    x::T
    y::T
end
Base.:+(p::Point3, q::Point3) = Point3(p.x + q.x, p.y + q.y)
Base.zero{T}(::Type{Point3{T}}) = Point3(zero(T),zero(T))

Point3(3,4)

Point3{Int64}(3, 4)

Here, `Point3` is actually a family of subtypes `Point{T}` for different types `T`.   The notation `<:` in Julia means "is a subtype of", and hence `T<:Real` means that we are constraining `T` to be a `Real` type (a built-in *abstract type* in Julia that includes e.g. integers or floating point).

In [73]:
Point3(3,4) + Point3(5.6, 7.8)

Point3{Float64}(8.6, 11.8)

Now, let's make an array:

In [74]:
a3 = Point3.(a,a)

10000000-element Array{Point3{Float64},1}:
 Point3{Float64}(0.106974, 0.106974)  
 Point3{Float64}(0.188845, 0.188845)  
 Point3{Float64}(0.484844, 0.484844)  
 Point3{Float64}(0.885487, 0.885487)  
 Point3{Float64}(0.33817, 0.33817)    
 Point3{Float64}(0.663541, 0.663541)  
 Point3{Float64}(0.906624, 0.906624)  
 Point3{Float64}(0.601869, 0.601869)  
 Point3{Float64}(0.788574, 0.788574)  
 Point3{Float64}(0.75761, 0.75761)    
 Point3{Float64}(0.225378, 0.225378)  
 Point3{Float64}(0.234817, 0.234817)  
 Point3{Float64}(0.403443, 0.403443)  
 ⋮                                    
 Point3{Float64}(0.410147, 0.410147)  
 Point3{Float64}(0.0750395, 0.0750395)
 Point3{Float64}(0.060309, 0.060309)  
 Point3{Float64}(0.307107, 0.307107)  
 Point3{Float64}(0.559202, 0.559202)  
 Point3{Float64}(0.498393, 0.498393)  
 Point3{Float64}(0.663843, 0.663843)  
 Point3{Float64}(0.441784, 0.441784)  
 Point3{Float64}(0.331159, 0.331159)  
 Point3{Float64}(0.0522143, 0.0522143)
 Point3{Float64}(0.38

Note that the type of this array is `Array{Point3{Float64},1}` (we could equivalently write this as `Vector{Point3{Float64}}`, since `Vector{T}` is a synonym for `Array{T,1}`).  You should learn a few things from this:

* An `Array{T,N}` in Julia is itself a parameterized type, parameterized by the element type `T` and the dimensionality `N`.

* Since the element type `T` is encoded in the `Array{T,N}` type, the element type does not need to be stored in each element.  That means that the `Array` is free to store an array of "inlined" elements, rather than an array of pointers to boxes.  (This is why `Array{Float64,1}` earlier could be stored in memory like a C `double*`.

* It is still important that the element type be `immutable`, since an array of mutable elements would still need to be an array of pointers (so that it could "notice" if another reference to an element mutates it).

In [75]:
@btime sum($a3)
@btime mysum($a3)

  12.744 ms (0 allocations: 0 bytes)
  11.766 ms (0 allocations: 0 bytes)


Point3{Float64}(5.000392958354606e6, 5.000392958354606e6)

Hooray! It is again **only about 10ms**, the same time as our completely concrete and inflexible `Point2`.